In [1]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from copy import deepcopy
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [ ]:
zipsM= Draft(player_ranking_zips, number_teams = 5, roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':2,'RP':2,'P':2,'BN':1})
zipsM.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [2, 68.5]
Stored Result for Pick 1 Mike Trout UTIL whose score is [2, 67.5]
Stored Result for Pick 2 Ronald Acuña BN whose score is [4, 63.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [3, 39.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [3, 41.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [3, 40.0]
Stored Result for Pick 6 Justin Verlander SP whose score is [3, 42.5]
Stored Result for Pick 7 Jacob deGrom P whose score is [3, 45.0]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [2, 62.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [5, 50.5]
Stored Result for Pick 10 J.T. Realmuto C whose score is [5, 54.5]
Team 1 picking Mookie Betts for OF
Stored Result for Pick 0 Alex Bregman 3B whose score is [2, 69.0]
Stored Result for Pick 1 Nolan Arenado UTIL whose score is [2, 68.5]
Stored Result for Pick 2 Justin Verlander SP whose score is [2, 67.0]
Stored Result for Pick

In [246]:
zipsM.roto_placement

1

In [239]:
zipsM.roto_standings

1    72.0
0    70.0
4    69.0
3    49.5
2    39.5
dtype: float64

In [ ]:
zipsS= Draft(player_ranking_zips, number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})

In [247]:
steamM= Draft(player_ranking_zips, number_teams = 6, roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2})
steam = Draft(player_ranking_steam)

In [248]:
steam.draft_all()

Picking from:
                PLAYER EligiblePosition
Rank                                   
2         Mookie Betts               OF
3           Mike Trout               OF
4         Ronald Acuña               OF
6       Anthony Rendon               3B
7       Cody Bellinger            1B/OF
9         Alex Bregman            3B/SS
11    Justin Verlander               SP
13        Jacob deGrom               SP
15     Whit Merrifield            2B/OF
27       Roberto Osuna               RP
185      J.T. Realmuto                C
Stored Result for Pick 0 Mookie Betts OF whose score is [9, 108.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [5, 127.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [11, 146.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [8, 159.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [7, 81.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [9, 113.5]
Stored Result for Pick 6 Justin Verlander SP whose score is 

In [255]:
steam.roto_team_stats.rank()

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,7.0,6.0,6.0,12.0,8.0,8.0,12.0,7.0,11.0,11.0,11.0,12.0,11.5,11.5,5.0,11.0,11.0,10.0,8.0,5.5
1,10.0,8.0,9.5,6.0,12.0,12.0,10.0,6.0,6.0,10.0,10.0,10.0,9.5,9.5,3.0,10.0,10.0,7.0,5.0,3.0
2,8.0,9.0,7.0,4.0,5.5,6.0,8.5,9.0,9.0,5.0,5.5,8.0,6.5,6.5,9.0,4.0,2.0,12.0,11.0,9.5
3,2.0,1.0,1.0,1.0,3.0,2.0,2.0,4.0,3.0,12.0,12.0,11.0,11.5,11.5,11.0,12.0,12.0,5.0,6.0,11.0
4,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,12.0,6.0,9.0,9.0,6.5,6.5,4.0,9.0,9.0,6.0,7.0,3.0
5,3.0,3.0,3.0,3.0,2.0,3.0,3.0,3.0,5.0,9.0,2.0,5.0,2.0,2.0,12.0,6.0,3.0,8.0,9.0,12.0
6,6.0,7.0,9.5,6.0,10.5,9.0,5.0,8.0,7.0,8.0,8.0,6.0,4.0,4.0,1.0,5.0,7.0,3.0,4.0,1.0
7,5.0,5.0,5.0,10.5,5.5,5.0,6.0,5.0,10.0,7.0,3.0,7.0,2.0,2.0,7.0,8.0,6.0,11.0,12.0,7.0
8,9.0,10.0,8.0,9.0,7.0,7.0,11.0,12.0,4.0,3.0,4.0,1.5,9.5,9.5,2.0,3.0,4.0,2.0,3.0,3.0
9,12.0,12.0,12.0,10.5,10.5,11.0,7.0,11.0,8.0,4.0,1.0,1.5,2.0,2.0,10.0,1.0,1.0,9.0,10.0,9.5


In [251]:
steam.teams[1]['roster']

{'C': array(['J.T. Realmuto'], dtype=object),
 'P': [[array(['Walker Buehler'], dtype=object),
   array(['Chris Paddack'], dtype=object)],
  array(['MacKenzie Gore'], dtype=object)],
 '3B': array(['Rafael Devers'], dtype=object),
 'SP': [array(['Mike Clevinger'], dtype=object),
  array(['Shane Bieber'], dtype=object)],
 'SS': array(['Marcus Semien'], dtype=object),
 'OF': [[array(['Starling Marte'], dtype=object),
   array(['Victor Robles'], dtype=object)],
  array(['Giancarlo Stanton'], dtype=object)],
 'UTIL': array(['Josh Donaldson'], dtype=object),
 '1B': array(['Trey Mancini'], dtype=object),
 '2B': array(['Gleyber Torres'], dtype=object),
 'RP': [array(['Trevor May'], dtype=object),
  array(['Brusdar Graterol'], dtype=object)],
 'BN': [[[[array(['César Hernández'], dtype=object),
     array(['Chris Bassitt'], dtype=object)],
    array(['Carl Edwards'], dtype=object)],
   array(['Cam Bedrosian'], dtype=object)],
  array(['Matt Magill'], dtype=object)]}

In [203]:
zips.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [2, 79.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [2, 79.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [2, 77.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [2, 75.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [3, 76.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [1, 77.0]
Stored Result for Pick 6 Justin Verlander SP whose score is [3, 78.0]
Stored Result for Pick 7 Jacob deGrom P whose score is [3, 78.5]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [3, 83.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [3, 84.0]
Stored Result for Pick 10 J.T. Realmuto C whose score is [2, 83.5]
PLAYER              Best Pick is Alex Bregman
EligiblePosition           Best Pick is 3B/SS
Name: 9, dtype: object
Forcing Pick
Team 1 force BEST pick Alex Bregman for SS
Stored Result for Pick 0 Justin Verlander SP whose score is [1, 77.0]
Stored Result for Pic

In [213]:
zips.teams[1]['roster']

{'1B': array(['Cody Bellinger'], dtype=object),
 'SP': [array(['Walker Buehler'], dtype=object),
  array(['Zack Greinke'], dtype=object)],
 'SS': array(['Trea Turner'], dtype=object),
 'BN': [[[[array(['Stephen Strasburg'], dtype=object),
     array(['Trevor May'], dtype=object)],
    array(['Matt Manning'], dtype=object)],
   array(['César Hernández'], dtype=object)],
  array(['Chris Bassitt'], dtype=object)],
 '2B': array(['Jeff McNeil'], dtype=object),
 '3B': array(['Eduardo Escobar'], dtype=object),
 'RP': [array(['Yusmeiro Petit'], dtype=object),
  array(['Rafael Montero'], dtype=object)],
 'P': [[array(['Blake Snell'], dtype=object),
   array(['Tommy Kahnle'], dtype=object)],
  array(['Lucas Giolito'], dtype=object)],
 'UTIL': array(['Gleyber Torres'], dtype=object),
 'C': array(['J.T. Realmuto'], dtype=object),
 'OF': [[array(['Nick Senzel'], dtype=object),
   array(['David Fletcher'], dtype=object)],
  array(['Willie Calhoun'], dtype=object)]}

In [259]:
zips.teams[1]['roster']

{'C': array(['J.T. Realmuto'], dtype=object),
 'UTIL': array(['Yordan Alvarez'], dtype=object),
 '3B': array(['Rafael Devers'], dtype=object),
 '1B': array(['Carlos Santana'], dtype=object),
 'SP': [array(['Tyler Glasnow'], dtype=object),
  array(['Chris Paddack'], dtype=object)],
 'SS': array(['Marcus Semien'], dtype=object),
 'OF': [[array(['Starling Marte'], dtype=object),
   array(['Trey Mancini'], dtype=object)],
  array(['David Peralta'], dtype=object)],
 'BN': [[[[array(['Josh Donaldson'], dtype=object),
     array(['Matt Manning'], dtype=object)],
    array(['Chris Bassitt'], dtype=object)],
   array(['Julio Urías'], dtype=object)],
  array(['Marcus Walden'], dtype=object)],
 '2B': array(['Gleyber Torres'], dtype=object),
 'RP': [array(['Yonny Chirinos'], dtype=object),
  array(['Trevor May'], dtype=object)],
 'P': [[array(['MacKenzie Gore'], dtype=object),
   array(['Dellin Betances'], dtype=object)],
  array(['Kyle Hendricks'], dtype=object)]}

In [257]:
zips.roto_placement

1

In [256]:
zips = Draft(player_ranking_zips)
zips.draft_all()

Picking from:
                PLAYER EligiblePosition
Rank                                   
2         Mookie Betts               OF
3           Mike Trout               OF
4         Ronald Acuña               OF
6       Anthony Rendon               3B
7       Cody Bellinger            1B/OF
9         Alex Bregman            3B/SS
11    Justin Verlander               SP
13        Jacob deGrom               SP
15     Whit Merrifield            2B/OF
27       Roberto Osuna               RP
185      J.T. Realmuto                C
Stored Result for Pick 0 Mookie Betts OF whose score is [8, 177.5]
Stored Result for Pick 1 Mike Trout UTIL whose score is [5, 132.5]
Stored Result for Pick 2 Ronald Acuña BN whose score is [9, 119.5]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [6, 129.5]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [6, 139.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [6, 131.0]
Stored Result for Pick 6 Justin Verlander SP whose score is 